# A simple RAG application using open-source models

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#MODEL = "gpt-3.5-turbo"
#MODEL = "phi3"
MODEL = "llama3"


In [2]:
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me an intersting fact")

/Users/juan.quiroga/Desktop/Estudio/MAIN/GIT/LLMops/LLMops/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


'Here\'s one:\n\nDid you know that there is a type of jellyfish that is immortal?!\n\nThe Turritopsis dohrmani, also known as the "immortal jellyfish," is a species of jellyfish that can transform its body into a younger state through a process called transdifferentiation. This means that it can essentially revert back to its polyp stage, which is the juvenile form of a jellyfish, and then grow back into an adult again!\n\nThis unique ability has fascinated scientists, who are studying this species to learn more about how it works and if it could potentially lead to new discoveries in fields like regenerative medicine.\n\nIsn\'t that just mind-blowing?\n\nWould you like to hear another interesting fact?'

In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me an intersting fact")

'Here\'s one:\n\nDid you know that there is a species of jellyfish that is immortal? Yes, you read that right! The Turritopsis dohrnii, also known as the "immortal jellyfish," is a type of jellyfish that can transform its body into a younger state through a process called transdifferentiation. This means that it can essentially revert back to its polyp stage, which is the juvenile form of a jellyfish, and then grow back into an adult again. This process can be repeated indefinitely, making the Turritopsis dohrnii theoretically immortal!\n\nIsn\'t that cool?'

In [4]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [5]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Juan", "question": "What's your name'?"})

'My name is Juan.'

In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("El Viejo y El Mar.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='- 2 -    Ernest  Hemi ngway \n \n \n \n \n \n \n \nEL VIEJO Y EL MAR \nErnest Hemingway  \n \n \n \n \n \n \n \n \nTítulo original: The Old Man and the Sea \n1ª Edición: Colección "Dante Quincenal": 1989 \n© Producción Editorial Dante, S. A. de C. V. Calle 59 № 548–7, C. P. 97000 Mérida, Yucatán, \nMéxico \nQueda hecho el depósito que marca la ley. \nI. S. B. N. 968–7232–05–6 \nDiseño de portada: Carlos Cámara Ruz, basado en fotografía de San Haskins \nIMPRESO EN MEXICO PRINTED IN MÉXICO \n \nEste libro fue digitalizado para su distribución gratuita y sin fines de lucro.', metadata={'source': 'El Viejo y El Mar.pdf', 'page': 1}),
 Document(page_content='El viejo y el mar  - 3 - \nEra un viejo que pescaba solo  en un bote en el Gulf Stream y hacía ochenta y \ncuatro días que no cogía un pez. En lo s primeros cuarent a días había tenido \nconsigo a un muchacho. Pero después de cuarenta días sin haber pescado los \npadres del muchacho le habían dicho que el viejo e

In [7]:
# to save the vectors in the cache memory, and store the artefacts
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

import pickle

# Save documents
with open('documents.pkl', 'wb') as f:
    pickle.dump(pages, f)

# Save embeddings
with open('embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)


In [8]:
# import pickle
# from langchain_community.vectorstores import DocArrayInMemorySearch

# # Load documents
# with open('documents.pkl', 'rb') as f:
#     pages = pickle.load(f)

# # Load embeddings
# with open('embeddings.pkl', 'rb') as f:
#     embeddings = pickle.load(f)

# # Reconstruct the vector store
# vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)


In [9]:
retriever = vectorstore.as_retriever()
retriever.invoke("pescar")

[Document(page_content='El viejo y el mar  - 13 - \n–Lo mismo yo –dijo el muchacho–. Ahora voy a buscar sus sardinas y las mías \ny sus carnadas frescas. El dueño trae el mismo nuestro aparejo. No quiere nunca \nque nadie lleve nada. \n–Somos diferentes –dijo el viejo–. Yo  te dejaba llevar las cosas cuando tenías \ncinco años. \n–Lo sé –dijo el muchacho–. Vuelvo ensegui da. Tome otro café. Aquí tenemos \ncrédito.  \nSalió, descalzo, por las rocas de coral hasta la nevera donde se guardaban las \ncarnadas. \nEl viejo tomó lentamente su café. Era lo único que tomaría en todo el día y \nsabía que debía tomarlo. Hacía mucho ti empo que le mortificaba comer y jamás \nllevaba un almuerzo. Tenía una botella de agua en la proa del bote y eso era lo \núnico que necesitaba para todo el día. \nEl muchacho estaba de vuelta con las sardinas y las dos carnadas envueltas en \nun periódico y bajaron por la vereda has ta el bote, sintiendo la arena con \npiedrecitas debajo de los pies, y levant aron e

In [10]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [11]:
questions = [
    "What is the purpose of the book?",
    "Que relacion tiene el viejo con el mar?",
    "are there other carachters different than the old man?",
    "que habla el libro sobre la muerte?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is the purpose of the book?


Answer: Based on the provided context, it appears that the purpose of the book "El Viejo y El Mar" (The Old Man and the Sea) is to tell a story about an old man who goes out to sea and catches a giant fish. The story explores themes of perseverance, loneliness, and the human struggle against nature.

In particular, the context suggests that the book may be exploring the idea of a person's connection with nature, as well as their own mortality and limitations. The old man's journey is likely meant to serve as a symbol for the universal human experience of facing challenges and overcoming obstacles.

The story seems to be focusing on the old man's emotional and psychological state as he struggles to come to terms with his own aging and the physical demands of fishing. The book may also be exploring themes of existential crisis, as the old man grapples with the meaning of life and his place in the world.

Overall, the purpose of the book appears to be to create a narrative that explores t

In [12]:
chain.batch([{"question": q} for q in questions])

['Based on the context provided, it appears that the purpose of the book "El Viejo y El Mar" (The Old Man and the Sea) by Ernest Hemingway is to tell a story about an old man\'s struggle with nature and his own mortality.\n\nThe passage describes the old man\'s experience as he attempts to catch a giant fish, which represents a symbol of power, beauty, and life. The story explores themes such as perseverance, self-discovery, and the human condition, as the old man faces challenges and reflects on his experiences.\n\nOverall, the purpose of the book seems to be to explore the complexities of the human experience through the eyes of an old man who is struggling to come to terms with his own mortality and the natural world around him.',
 'Según el contexto, el viejo ha pasado la mayor parte de su vida en el mar, por lo que se puede inferir que él tiene una relación muy estrecha y profunda con el mar. La narrativa sugiere que el viejo es un anciano experimentado que ha pasado muchos años n

In [13]:
for s in chain.stream({"question": "que habla el libro sobre la muerte?"}):
    print(s, end="", flush=True)

Basado en el contexto proporcionado, no hay una respuesta directa a esta pregunta en los extractos dados. Sin embargo, se puede inferir que el libro "El Viejo y El Mar" podría abordar la muerte de manera metafórica o simbólica, ya que el pez capturado por el viejo puede representar la lucha interna con la mortalidad.

En el extracto de página 5, se describe a un pez "con la muerte en la entraña" que es atrapado y muere después. Esto puede ser visto como una metáfora para la muerte, ya que el viejo siente que él mismo está cerca de morir.

En el extracto de página 50, se describe a un pez "plat eado y quieto" flotando en el agua, lo que puede representar la tranquilidad o paz que puede seguir a la muerte. El viejo también se siente cansado y viejo, lo que sugiere que él mismo está enfrentando la mortalidad.

En general, aunque no hay una respuesta directa sobre la muerte, es probable que el libro "El Viejo y El Mar" aborde esta temática de manera simbólica o metafórica.